In [2]:
!pip install mlxtend

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 10.5 MB/s eta 0:00:0000:0100:01


In [1]:
import pandas as pd
import numpy as np
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori, association_rules

In [2]:
df = pd.read_csv('해시태그완성데이터.csv')
df.head()

,관광지명,광역시/도,시/군/구,도로명주소,중분류 카테고리,소분류 카테고리,검색건수,관광지,언급량,관광지명.1,해시태그,가족여부,연인여부,친구여부,아이여부,휴식,교육
0,아산스파비스,충청남도,아산시,충남 아산시 아산온천로157번길 67-0,체험관광,웰니스관광,49046,아산스파비스,1249,아산스파비스,"#가족여행, #관광지, #스파, #아산스파비스, #아이와함께, #연인과함께, #온천...",1,1,0,1,1,0
1,파라다이스스파도고,충청남도,아산시,충남 아산시 도고온천로 176-1,체험관광,웰니스관광,35164,파라다이스스파도고,1055,파라다이스스파도고,"#2023_설날_온천, #가족과함께, #가족여행, #관광지, #데이트코스, #뷰티&...",1,1,0,0,1,0
2,하늘물빛정원,충청남도,금산군,충남 금산군 검한1길 156-0,체험관광,농/산/어촌체험,22495,하늘물빛정원,738,하늘물빛정원,"#가족여행, #걷기길, #관광지, #꽃여행, #먹거리, #미식여행, #사계절, #산...",1,0,0,0,1,0
3,팜카밀레허브농원,충청남도,태안군,충남 태안군 우운길 56-19,체험관광,농/산/어촌체험,21646,팜카밀레허브농원,515,팜카밀레허브농원,"#체험, #팜카밀레허브농원, #충청권, #이색체험, #허브, #관광지, #자연환경,...",0,1,0,1,0,0
4,아그로랜드태신목장,충청남도,예산군,충남 예산군 상몽2길 231-0,체험관광,농/산/어촌체험,20323,아그로랜드태신목장,1050,아그로랜드태신목장,"#2022_추석_당일치기_여행, #가족체험, #관광지, #낙농체험, #아그로랜드태신...",1,0,0,1,1,0


In [3]:
df['해시태그'][0]

'#가족여행, #관광지, #스파, #아산스파비스, #아이와함께, #연인과함께, #온천, #온천여행, #워터파크, #충청권, #휴식공간, #휴식여행, #휴식하기, #휴식하기좋은곳, #휴양여행, #힐링, #힐링여행'

In [4]:
df['해시태그'][1]

'#2023_설날_온천, #가족과함께, #가족여행, #관광지, #데이트코스, #뷰티&스파, #스파, #실내수영장, #야외수영장, #연인과함께, #온천&스파, #온천여행, #이색체험, #충청권, #파라다이스스파도고, #휴식공간, #휴식여행, #힐링, #힐링&휴양여행'

In [5]:
hashtags = [
    ['#가족여행', '#관광지', '#스파', '#아산스파비스', '#아이와함께', '#연인과함께', '#온천', '#온천여행', '#워터파크', '#충청권', '#휴식공간', '#휴식여행', '#휴식하기', '#휴식하기좋은곳', '#휴양여행', '#힐링', '#힐링여행'],
    ['#2023_설날_온천', '#가족과함께', '#가족여행', '#관광지', '#데이트코스', '#뷰티&스파', '#스파', '#실내수영장', '#야외수영장', '#연인과함께', '#온천&스파', '#온천여행', '#이색체험', '#충청권', '#파라다이스스파도고', '#휴식공간', '#휴식여행', '#힐링', '#힐링&휴양여행']
]

# TransactionEncoder를 사용하여 아이템 집합 생성
te = TransactionEncoder()
te_array = te.fit_transform(hashtags)
pd.DataFrame(te_array, columns=te.columns_)

,#2023_설날_온천,#가족과함께,#가족여행,#관광지,#데이트코스,#뷰티&스파,#스파,#실내수영장,#아산스파비스,#아이와함께,...,#충청권,#파라다이스스파도고,#휴식공간,#휴식여행,#휴식하기,#휴식하기좋은곳,#휴양여행,#힐링,#힐링&휴양여행,#힐링여행
0,False,False,True,True,False,False,True,False,True,True,...,True,False,True,True,True,True,True,True,False,True
1,True,True,True,True,True,True,True,True,False,False,...,True,True,True,True,False,False,False,True,True,False


## 해시태그 데이터 정리

In [10]:
import pandas as pd

# 해시태그, 콤마, 공백을 공백 한 칸으로 전환하는 함수
def clean_hashtags(text):
    text = text.replace('#', '')  # 해시태그를 공백으로 대체
    text = text.replace(',', '')  # 콤마를 공백으로 대체
    text = text.replace('관광지', '') # 관광지 해시태그 제거
    text = text.replace('충청권', '') # 관광지 해시태그 제거
    text = ' '.join(text.split())  # 공백을 한 칸으로 조정
    return text

# 데이터프레임의 해시태그 컬럼에 적용하여 값 변경
df['hashtags'] = df['해시태그'].apply(clean_hashtags)


In [11]:
def extract_hashtags(text):
    hashtags = text.split()
    return hashtags

text = '2023_설날_온천 가족과함께 가족여행 관광지 데이트코스 뷰티&스파 스파 실내수영장 야외수영장 연인과함께 온천&스파 온천여행 이색체험 충청권 파라다이스스파도고 휴식공간 휴식여행 힐링 힐링&휴양여행'

hashtags_list = extract_hashtags(text)
print(hashtags_list)

['2023_설날_온천', '가족과함께', '가족여행', '관광지', '데이트코스', '뷰티&스파', '스파', '실내수영장', '야외수영장', '연인과함께', '온천&스파', '온천여행', '이색체험', '충청권', '파라다이스스파도고', '휴식공간', '휴식여행', '힐링', '힐링&휴양여행']


In [12]:
tag_lists = []
for i in range(len(df)):
    tags = extract_hashtags(df['hashtags'][i])
    tag_lists.append(tags)

In [13]:
tag_lists

[['가족여행',
  '스파',
  '아산스파비스',
  '아이와함께',
  '연인과함께',
  '온천',
  '온천여행',
  '워터파크',
  '휴식공간',
  '휴식여행',
  '휴식하기',
  '휴식하기좋은곳',
  '휴양여행',
  '힐링',
  '힐링여행'],
 ['2023_설날_온천',
  '가족과함께',
  '가족여행',
  '데이트코스',
  '뷰티&스파',
  '스파',
  '실내수영장',
  '야외수영장',
  '연인과함께',
  '온천&스파',
  '온천여행',
  '이색체험',
  '파라다이스스파도고',
  '휴식공간',
  '휴식여행',
  '힐링',
  '힐링&휴양여행'],
 ['가족여행',
  '걷기길',
  '꽃여행',
  '먹거리',
  '미식여행',
  '사계절',
  '산책길',
  '산책하기',
  '산책하기좋은곳',
  '야경투어',
  '자연&숲치유',
  '캠핑&글램핑',
  '하늘물빛정원',
  '휴식공간',
  '휴식여행',
  '휴식하기',
  '휴식하기좋은곳',
  '힐링&휴양여행'],
 ['체험',
  '팜카밀레허브농원',
  '이색체험',
  '허브',
  '자연환경',
  '자연좋은곳',
  '아이와함께',
  '연인과함께',
  '남녀노소',
  '사진찍기좋은곳',
  '반려견동반여행지'],
 ['2022_추석_당일치기_여행',
  '가족체험',
  '낙농체험',
  '아그로랜드태신목장',
  '아이가_좋아하는_여행',
  '아이와함께',
  '이색체험',
  '자연환경',
  '체험',
  '체험프로그램',
  '휴식여행',
  '휴식하기좋은곳'],
 ['수목원아이와함께데이트코스체험학습이색체험자연사진찍기사진찍기좋은곳휴식하기좋은곳휴식여행휴식공간'],
 ['가족여행',
  '갯벌체험',
  '갯벌체험마을',
  '생태체험학습장',
  '선도리갯벌체험마을',
  '쌍도',
  '아이와함께',
  '자연속으로',
  '자연여행',
  '자연환경',
  '체험',
  '휴식공간',
  '휴식여행',
  '휴식하

## 연관분석 진행

- 지지도 (support) : 전체 항목 중(행들 중) x와 y를 모두 포함하는 경우의 비율 (전체 데이터에서 해당 해시태그가 포함된 개수의 비율)
    -  연관 규칙의 신뢰도와 마찬가지로 0부터 1 사이의 값을 가지며, 높을수록 해당 항목 집합이 더 빈번하게 발생한다
- 신뢰도 (Confidence) : 조건절이 발생했을 때, 결과절도 함께 발생할 확률을 나타냅니다. 즉, 어떤 항목 집합이 주어졌을 때 다른 항목 집합이 발생할 확률을 의미
    - ex, "#가족여행"이라는 해시태그를 포함하는 거래 중에서 "#관광지"라는 해시태그도 함께 포함되는 경우

- 향상도 (lift) : 두 항목 간의 상호 등장 패턴을 측정합니다. 특정 항목 집합이 발생했을 때 결과 항목이 발생할 확률을 기대되는 발생 확률에 대한 비율
    - 향상도가 1보다 크면 두 항목 간의 양의 상관관계가 있다고 해석할 수 있습니다. 즉, 조건절과 결과절이 함께 발생할 확률이 서로 독립적인 경우보다 높다는 것을 의미합니다. 이는 두 항목 간의 관련성이 있고, 하나의 항목이 다른 항목의 발생을 유발한다는 것을 나타냅니다.

In [14]:
te = TransactionEncoder()
te_ary = te.fit(tag_lists).transform(tag_lists)
tag_df = pd.DataFrame(te_ary, columns=te.columns_)

In [15]:
tag_df

,15_16한국관광100선,19_20한국관광100선,2020_휴가철찾아가고싶은_33섬,2020비대면_가을편,2021대한민국안심여행캠페인,2022_추석_당일치기_여행,2022대한민국안심여행캠페인,2022찾아가고싶은섬,2022찾아가고싶은섬_여름섬,2023_설날_온천,...,휴식하기,휴식하기좋은곳,휴양림,휴양여행,힐링,힐링&여행,힐링&휴양,힐링&휴양여행,힐링산책,힐링여행
0,False,False,False,False,False,False,False,False,False,False,...,True,True,False,True,True,False,False,False,False,True
1,False,False,False,False,False,False,False,False,False,True,...,False,False,False,False,True,False,False,True,False,False
2,False,False,False,False,False,False,False,False,False,False,...,True,True,False,False,False,False,False,True,False,False
3,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,True,False,False,False,False,...,False,True,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
402,False,False,False,False,False,False,False,False,False,False,...,True,True,False,False,False,False,False,False,False,False
403,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,True,False,False,False,False,False
404,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
405,False,True,False,False,True,False,False,False,False,False,...,False,False,False,False,True,False,False,False,False,False


In [29]:
# 지지도가 0.5 이상인 것들 파악
frequent_itemsets = apriori(tag_df, min_support=0.12, use_colnames=True)

In [30]:
frequent_itemsets

,support,itemsets
0,0.199017,(가족여행)
1,0.245700,(아이와함께)
2,0.191646,(역사)
3,0.154791,(연인과함께)
4,0.240786,(자연)
5,0.248157,(자연좋은곳)
6,0.191646,(자연환경)
7,0.176904,(친구와함께)
8,0.142506,(휴식공간)
9,0.174447,(휴식여행)


In [31]:
# 신뢰도
association_rules(frequent_itemsets, metric="confidence", min_threshold=0.5)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
0,(아이와함께),(가족여행),0.245700,0.199017,0.132678,0.540000,2.713333,0.083780,1.741267,0.837134
1,(가족여행),(아이와함께),0.199017,0.245700,0.132678,0.666667,2.713333,0.083780,2.262899,0.788344
2,(자연좋은곳),(아이와함께),0.248157,0.245700,0.127764,0.514851,2.095446,0.066792,1.554781,0.695324
3,(아이와함께),(자연좋은곳),0.245700,0.248157,0.127764,0.520000,2.095446,0.066792,1.566339,0.693059
4,(아이와함께),(친구와함께),0.245700,0.176904,0.125307,0.510000,2.882917,0.081842,1.679787,0.865875
5,(친구와함께),(아이와함께),0.176904,0.245700,0.125307,0.708333,2.882917,0.081842,2.586171,0.793503
6,(연인과함께),(친구와함께),0.154791,0.176904,0.122850,0.793651,4.486332,0.095467,3.988849,0.919419
7,(친구와함께),(연인과함께),0.176904,0.154791,0.122850,0.694444,4.486332,0.095467,2.766138,0.944119
8,(자연좋은곳),(자연),0.248157,0.240786,0.142506,0.574257,2.384926,0.082753,1.783270,0.772369
9,(자연),(자연좋은곳),0.240786,0.248157,0.142506,0.591837,2.384926,0.082753,1.842015,0.764870


In [32]:
# 향상도
association_rules(frequent_itemsets, metric="lift", min_threshold=1)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
0,(아이와함께),(가족여행),0.245700,0.199017,0.132678,0.540000,2.713333,0.083780,1.741267,0.837134
1,(가족여행),(아이와함께),0.199017,0.245700,0.132678,0.666667,2.713333,0.083780,2.262899,0.788344
2,(자연좋은곳),(아이와함께),0.248157,0.245700,0.127764,0.514851,2.095446,0.066792,1.554781,0.695324
3,(아이와함께),(자연좋은곳),0.245700,0.248157,0.127764,0.520000,2.095446,0.066792,1.566339,0.693059
4,(아이와함께),(친구와함께),0.245700,0.176904,0.125307,0.510000,2.882917,0.081842,1.679787,0.865875
5,(친구와함께),(아이와함께),0.176904,0.245700,0.125307,0.708333,2.882917,0.081842,2.586171,0.793503
6,(연인과함께),(친구와함께),0.154791,0.176904,0.122850,0.793651,4.486332,0.095467,3.988849,0.919419
7,(친구와함께),(연인과함께),0.176904,0.154791,0.122850,0.694444,4.486332,0.095467,2.766138,0.944119
8,(자연좋은곳),(자연),0.248157,0.240786,0.142506,0.574257,2.384926,0.082753,1.783270,0.772369
9,(자연),(자연좋은곳),0.240786,0.248157,0.142506,0.591837,2.384926,0.082753,1.842015,0.764870


In [98]:
frequent_itemset = apriori(tag_df,
                           min_support=0.05, 
                           max_len=3, 
                           use_colnames=True, 
                           verbose=1 
                          )
frequent_itemset.sort_values(by='support', ascending=False, inplace=True)

Processing 9579 combinations | Sampling itemset size 3


In [99]:
frequent_itemset

,support,itemsets
13,0.368550,(아이와함께)
15,0.319410,(역사)
1,0.289926,(가족여행)
32,0.287469,(자연좋은곳)
29,0.275184,(자연)
...,...,...
151,0.051597,"(자연속으로, 역사)"
474,0.051597,"(역사, 사찰, 전통사찰)"
139,0.051597,"(자연, 여름여행)"
63,0.051597,"(기차여행, 친구와함께)"


In [52]:
frequent_itemset.reset_index(drop=True, inplace=True)

In [59]:
frequent_itemset['itemsets'][2]

frozenset({'관광지', '충청권'})

In [68]:
# 콤마가 포함된 항목들만 추출하는 함수
def extract_comma_items(frozenset_items):
    comma_items = []
    for item in frozenset_items:
        item_str = str(item)
        if ',' in item_str:
            comma_items.append(item_str)
    return comma_items

# 콤마가 포함된 항목들만 추출
extracted_items = extract_comma_items(frequent_itemset['itemsets'])

print(extracted_items)

["frozenset({'관광지', '충청권'})", "frozenset({'역사', '관광지'})", "frozenset({'관광지', '자연좋은곳'})", "frozenset({'자연', '관광지'})", "frozenset({'관광지', '아이와함께'})", "frozenset({'충청권', '아이와함께'})", "frozenset({'휴식하기좋은곳', '관광지'})", "frozenset({'가족여행', '충청권'})", "frozenset({'관광지', '자연환경'})", "frozenset({'가족여행', '관광지'})", "frozenset({'가족여행', '아이와함께'})", "frozenset({'관광지', '휴식여행'})", "frozenset({'휴식하기좋은곳', '휴식여행'})", "frozenset({'휴식하기좋은곳', '관광지', '휴식여행'})", "frozenset({'자연', '관광지', '충청권'})", "frozenset({'자연', '충청권'})", "frozenset({'친구와함께', '연인과함께'})", "frozenset({'친구와함께', '아이와함께'})", "frozenset({'친구와함께', '충청권'})", "frozenset({'가족여행', '관광지', '충청권'})", "frozenset({'역사', '충청권'})", "frozenset({'가족여행', '친구와함께'})", "frozenset({'관광지', '휴식공간'})", "frozenset({'아이와함께', '체험학습'})", "frozenset({'휴식하기좋은곳', '휴식공간'})", "frozenset({'관광지', '친구와함께'})", "frozenset({'가족여행', '연인과함께'})", "frozenset({'관광지', '충청권', '아이와함께'})", "frozenset({'휴식하기좋은곳', '관광지', '휴식공간'})", "frozenset({'관광지', '연인과함께'})", "frozenset({'휴식공간', '휴식여행'})", "fro